# Deep Learning Project

**Group:** Songbird  
**Members:** Charlotte de Vries, Jiazhen Tang, Paulo Zirlis

In [ ]:
# Setup block (packages)
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import os



## 1. Project Overview



***

## 2. Dataset Description

The dataset includes high-resolution CT and MRI images captured from multiple patients, with each image labeled with the corresponding tumor type (e.g., glioma, meningioma, etc.). For this project we will focus solely on the **MRI** images for simplicity. The dataset's creator collected these data from different sources to assist researchers and healthcare professionals in developing AI models for the automatic detection, classification, and segmentation of brain tumors.

The images are divided as follows:
- Healty images: 2000
- Tumor images: 3000
    - Glioma: 
    - Meningioma: 
    - Pituitary: 
    - Tumor: 
- **Total of images:** 5000

Source: [Brain tumor multimodal image (Kaggle)](https://www.kaggle.com/datasets/murtozalikhon/brain-tumor-multimodal-image-ct-and-mri/data)

In [ ]:
# Load data

***

## 2. Neural Network Models

### 2.1 Custom CNN

The Convolutional Neural Network (CNN) model designed for this project consists of four convolutional blocks followed by a final block with pooling, dropout and fully connected layers. Each convolutional block has a convolution layer, batch normalization, activation function (ReLU) and max pooling. Early stopping was added to control for overfitting and underfitting. Batch normalization was used to improve training speed and stability. Dropout was included in the final block to further prevent overfitting. The model was compiled with the Adam optimizer, categorical cross-entropy loss function, and accuracy as the evaluation metric.

<br>

The architecture is as follows:

**Input and Data Augmentation**
- Input layer: shape (256, 256, 1)
- Data Augmentation: Random rotations and horizontal flips

**First Convolutional Block**
- Conv2d layer: 32 filters, 3x3 kernel, stride of 1, same padding
- Batch Normalization
- ReLU Activation
- MaxPooling2d layer: 2x2 pool size, stride of 2.

**Other Convolutional Blocks**
- same as the first block but with increasing number of filters (64, 128, 256)

**Classifier Head**
- Global Average Pooling layer
- Dense layer: 64 units, ReLU activation
- Dropout layer: 0.3 dropout rate
- Dense layer: 5 units (nº of classes), Softmax activation

In [ ]:
### CNN Architecture

# Seed for reproducibility
np.random.seed(42)

# Custom CNN
CNN = keras.Sequential([
    
    # Input
    layers.InputLayer(shape=[256, 256, 1]),
    
    # Data Augmentation
    layers.RandomFlip("horizontal"), # flip images horizontally
    layers.RandomRotation(0.1),      # rotate images randomly by 10%


    # First Convolutional Block
    layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPool2D(pool_size=2, strides=2),

    # Second Convolutional Block
    layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPool2D(pool_size=2, strides=2),

    # Third Convolutional Block
    layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPool2D(pool_size=2, strides=2),

    # Fourth Convolutional Block
    layers.Conv2D(filters=256, kernel_size=3, strides=1, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPool2D(pool_size=2, strides=2),

    # Classifier Head
    layers.GlobalAveragePooling2D(),
    layers.Dense(units=64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(units=5, activation='softmax')  # 5 classes
])

CNN.summary()

In [ ]:
### Train and Evaluate CNN

# Compile the model
CNN.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

# Define early stopping
early_stopping = EarlyStopping(
    min_delta = 0.001,
    patience = 20,
    restore_best_weights = True
)

# Fit the model
history = CNN.fit(
    train,
    validation_data = valid,
    batch_size = 32,
    epochs = 25,
    callbacks = [early_stopping],
    verbose = 1
)

### ADD EVALUATION METRICS AND VISUALIZATIONS HERE ###

***

## 2.2 Pre-trained Residual Network
Charlotte

#### **ResNet without finetuning**

#### Preprocessing data

In [ ]:
ds_train_c5_, ds_val_c5_ = image_dataset_from_directory(
    '/tmp/BrainTumorDataset',
    validation_split=0.2,
    subset='both',
    seed=42,
    image_size=(224,224),
    batch_size=32,
    label_mode='categorical'
)

#### Data preprocessing

In [ ]:
from tensorflow.keras.applications.resnet import preprocess_input

def preprocess(image, label):
    image = preprocess_input(image)
    return image, label

ds_train_c5 = ds_train_c5_.map(preprocess)
ds_val_c5 = ds_val_c5_.map(preprocess)

#### Data augmentation

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

def augment(image, label):
    image = data_augmentation(image)
    return image, label

ds_train_c5 = ds_train_c5_.map(augment).map(preprocess)

#### Create the model

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

## pretrained base
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.trainable = False   # Freeze weights

## attach head
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(5, activation='softmax')
])

#### Train the model

In [ ]:
## train model
model.compile(
    optimizer=Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_c5 = model.fit(
    ds_train_c5,
    validation_data=ds_val_c5,
    epochs=10
)

#### Visualizing the loss and accuracy

In [ ]:
import matplotlib.pyplot as plt

# # Loss
# plt.plot(history_c5.history['loss'], label='train_loss')
# plt.plot(history_c5.history['val_loss'], label='val_loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

# # Accuracy
# plt.plot(history_c5.history['accuracy'], label='train_accuracy')
# plt.plot(history_c5.history['val_accuracy'], label='val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.show()

#### **ResNet with finetuning**

#### Create the model

In [ ]:
# Unfreeze the model
base_model.trainable = True

# Freeze the first 140 layers "freeze"
for layer in base_model.layers[:140]:
    layer.trainable = False

## attach head
model_finetuned = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(5, activation='softmax')
])

#### Train the model

In [ ]:
from tensorflow.keras.optimizers import Adam

model_finetuned.compile(
    optimizer=Adam(1e-5),    
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_c5_finetune = model_finetuned.fit(
    ds_train_c5,
    validation_data=ds_val_c5,
    epochs=10
)

#### Visualizing the loss and accuracy after fine tuning

In [ ]:
# Loss
# plt.plot(history_c5_finetune.history['loss'], label='train_loss')
# plt.plot(history_c5_finetune.history['val_loss'], label='val_loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

# # Accuracy
# plt.plot(history_c5_finetune.history['accuracy'], label='train_accuracy')
# plt.plot(history_c5_finetune.history['val_accuracy'], label='val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.show()

***

## 2.3 Pre-trained Vision Transformer
Jiazhen